## Interaktionseffekte

Open in Colab: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/febse/econ2025-de/blob/main/10-Interaction-Effects.ipynb)


In [ ]:
# Load the data
stu <- read.csv("https://raw.githubusercontent.com/febse/data/main/econ/drinking.csv") %>%
  mutate(
    sex = ifelse(male == 1, "M", "F")
  )

# Print the first few rows
stu %>% head()

In [ ]:
# Plot the data and overlay the regression line for gpa ~ drinks


Schätzen Sie die folgenden Modelle und interpretieren Sie die Koeffizienten. 

- $\text{gpa}_i = \beta_0 + \beta_1 \text{drink}_i + e_i$
- $\text{gpa}_i = \beta_0 + \beta_1 \text{sexMale}_i + e_i$
- $\text{gpa}_i =\beta_0 + \beta_1 \text{drink}_i + \beta_2 \text{sexMale}_i + e_i$
- $\text{gpa}_i = \beta_0 + \beta_1 \text{drink}_i + \beta_2 \text{sexMale}_i +
\beta_3 \text{gpa}_i \cdot \text{sexMale}_i$

## Polynomiale Regression

Zuerst generieren wir Daten aus dem folgenden Modell:

$$
y_i = 2 + x_i + x^2 + e_i, e_i \sim N(0, 1)
$$


In [ ]:
## Create the simulated sample
sim_n <- 100

poly_dt <- tibble(
  x = runif(n = sim_n, min = -4, max = 3),
  y = 2 +  x +  x^2 + rnorm(n = sim_n, mean = 0, sd = 1)
)

poly_dt %>%
  ggplot(aes(x = x, y = y)) +
  geom_point()



2.  Schätzen Sie ein lineares Regressionsmodell mit einem linearen Term für $x$. Schauen Sie sich die Regressionsgerade an, als auch die Residuen. Was fällt Ihnen auf?

$$
y_i = \beta_0 + \beta_1 x_i, e_i \sim N(0, \sigma^2)
$$


In [ ]:
fit1_pred <- predict(
  fit1,
  interval = "confidence"
  ) %>%
  as_tibble() %>%
  bind_cols(poly_dt) %>%
  mutate(
    res = y - fit
  )

In [ ]:
fit1_pred %>%
  ggplot(aes(x = fit, y = res)) +
  geom_point() +
  geom_hline(yintercept = 0, alpha = 0.5, lty = 2) +
  labs(
    x = "Predicted",
    y = "Residual"
  )


3.  Schätzen Sie ein polynomiales Regressionsmodell mit einem quadratischen Term für $x$. Schauen Sie sich die Regressionsgerade an, als auch die Residuen. 

$$
y_i = \beta_0 + \beta_1 x_i + \beta_2 x^2_i + e_i, e_i \sim N(0, \sigma^2)
$$


In [ ]:
fit2_pred <- predict(fit2, interval = "confidence") %>%
  as_tibble() %>%
  bind_cols(poly_dt) %>%
  mutate(
    res = y - fit
  )

In [ ]:
fit2_pred %>%
  ggplot(aes(x = x, y = y)) +
  geom_point() +
  geom_line(
    aes(y = fit),
    color = "steelblue2"
  ) +
  geom_ribbon(
    aes(ymin = lwr, ymax = upr),
    alpha = 0.2,
    fill = "steelblue2"
  )

In [ ]:
fit2_pred %>%
  ggplot(aes(x = fit, y = res)) +
  geom_point() +
  geom_hline(yintercept = 0, alpha = 0.5, lty = 2) +
  labs(
    x = "Predicted",
    y = "Residual"
  )

4.  Schätzen Sie das folgende polynomiale Regressionsmodell:

$$
y_i = \beta_0 + \beta_1 x_i + \beta_2 x^2_i + \beta_3 x^3_i + \beta_4 x^4_i + e_i, e_i \sim N(0, \sigma^2)
$$

In [ ]:
fit3_pred <- predict(fit3, interval = "confidence") %>%
  as_tibble() %>%
  bind_cols(poly_dt) %>%
  mutate(
    res = y - fit
  )

In [ ]:
fit3_pred %>%
  ggplot(aes(x = x, y = y)) +
  geom_point() +
  geom_line(
    aes(y = fit),
    color = "steelblue2"
  ) +
  geom_ribbon(
    aes(ymin = lwr, ymax = upr),
    alpha = 0.2,
    fill = "steelblue2"
  )

In [ ]:
fit3_pred %>%
  ggplot(aes(x = fit, y = res)) +
  geom_point() +
  geom_hline(yintercept = 0, alpha = 0.5, lty = 2) +
  labs(
    x = "Predicted",
    y = "Residual"
  )

## Model choice

1.  $R^2$

$$
R^2 = 1 - \frac{RSS}{TSS} \\
RSS = \sum_{i = 1}^{n}(y_i - \hat{y}_i)^2 \\
TSS = \sum_{i = 1}^{n}(y_i - \bar{y})^2
$$

- Berechnen Sie $R^2$ für die Modelle aus den obigen Aufgaben 2, 3 und 4. Welches Modell hat das höchste $R^2$?

- Berechnen Sie $R^2$ für die Modelle mit Testdaten. Welches Modell hat das höchste $R^2$?


In [ ]:
sim_noconst <- tibble(
  y = 2 + rnorm(n = 100, mean = 0, sd = 1),
  x = runif(n = 100)
)

sim_noconst %>%
  ggplot(aes(x = x, y = y)) +
  geom_point()

2.  Adjusted $R^2$

$$
R^2_{ADJ} = 1 - \frac{n - 1}{n - p - 1} \frac{RSS}{TSS}
$$

3.  Information Criteria

Compute the Akaike information criterion (AIC) for the three models and choose the best one.

$$
\text{AIC} = n \log\left(\frac{RSS}{n}\right) + 2p + n + n \log(2\pi).
$$

In [ ]:
AIC(fit1)
AIC(fit2)
AIC(fit3)

In [ ]:
smpl <- sample(1:100, 20)
poly_dt_estim <- poly_dt[smpl, ]
poly_dt_test <- poly_dt[-smpl, ]

fit2_estim <- lm(y ~ poly(x, 4), data = poly_dt_estim)
fit5_test <- lm(y ~ poly(x, 4), data = poly_dt_test)

pred2 <- predict(fit2_estim, newdata = poly_dt_estim)
pred5 <- predict(fit2_estim, newdata = poly_dt_estim)